### Test status indication in jupyter notebook
This will be a brief demonstration of the indicator tools for jupyter notebook. The indicators in `labbench` include
* `panel`, which autogenerates a table of state values, and
* `log_progress`, which shows through the test mainloop.
* `linspace(...)` is a shortcut for `log_progress(np.linspace(...))`
* `range(...)` is a shortcut for `log_progress(range(...))`

`panel` is called automatically from within a notebook on calls to `log_progress`, `linspace`, or `range`.

To get started here is the emulated instrument setup.

In [1]:
from __future__ import print_function

import sys,os,time
if '..' not in sys.path:
    sys.path.insert(0,'..')
import labbench as lb
import numpy as np
import pandas as pd

class EmulatedInstrument(lb.emulated.EmulatedVISADevice):
    ''' This "instrument" makes mock data and instrument states to
        demonstrate we can show the process of setting
        up a measurement.
    '''
    class state (lb.Device.state):
        initiate_continuous = lb.Bool(command='INIT:CONT')
        output_trigger      = lb.Bool(command='OUTP:TRIG')
        sweep_aperture      = lb.Float(command='SWE:APER', min=20e-6,
                                       max=200e-3, help='time (in s)')
        frequency           = lb.Float(command='SENS:FREQ', min=10e6,
                                       max=18e9, step=1e-3, help='center frequency (in Hz)')

    def trigger(self):
        ''' This would tell the instrument to start a measurement
        '''
        time.sleep(.1)

    def fetch_trace(self, N=1001):
        ''' Generate N points of junk data as a pandas series.
        '''
        values = np.random.uniform(-1, 1, N)
        index = np.linspace(0, self.state.sweep_aperture, N)
        series = pd.Series(values, index=index, name='Voltage (V)')
        series.index.name = 'Time (s)'
        return series

Here is the example use of the indicators. They are all triggered automatically by the use of `lb.linspace`. The explicit call to `panel()` is also included.

`panel` does introspection on the code where it is called to identify `lb.Device` instances. It observes changes to the states of these devices and shows a continuously updating table as the test progresses.

`log_progress` shows a bar indicating the progress through a test loop.

In [2]:
with EmulatedInstrument() as inst,\
        EmulatedInstrument() as inst2,\
        EmulatedInstrument() as inst3,\
        lb.StatesToSQLite('test.db', overwrite=False) as db:
            
    # panel()

    db.observe([inst,inst2,inst3], changes=True, always='sweep_aperture')

    for inst.state.frequency in lb.linspace(10e6,100e6,31):
        inst2.state.frequency = inst.state.frequency
        inst3.state.frequency = inst.state.frequency
        
        inst.trigger()
        trace = inst.fetch_trace()
        db.append(power_GW='1.21', trace=trace)

A Jupyter Widget

A Jupyter Widget